# Counting Cryptic Crossword Grids

In [1]:
%load_ext nb_black

import json
import itertools
import numpy as np
from tqdm.notebook import tqdm, trange

<IPython.core.display.Javascript object>

In [2]:
GRID_LENGTH = 13  # FIXME: must be odd...?
MIN_WORD_LENGTH = 3  # Minimum allowable word length
ROUND_UP_UNCHECKED = True  # I.e. can a 7-letter word have 3 unchecked squares?

<IPython.core.display.Javascript object>

In [3]:
# For now, True is white, False is black. We will reverse this later.
grid = np.meshgrid(*[[0, 1] for _ in range(GRID_LENGTH)])
grid = [mesh.flatten() for mesh in grid]
grid = np.transpose(np.vstack(grid))
grid = grid.astype(bool)

<IPython.core.display.Javascript object>

In [4]:
def contains_run_length(row, length):
    """
    If True, `row` contains a run of True values of length equal to `length`
    E.g.
        row_contains_run_length([0, 1, 1], 1) returns False
        row_contains_run_length([0, 1, 1], 2) returns True
    """
    # This is why we have white and black flipped - to sum lengths of _whites_
    return any([sum(group) == length for _, group in itertools.groupby(row)])

<IPython.core.display.Javascript object>

In [5]:
# Compute all valid rows

for length in range(2, MIN_WORD_LENGTH):
    rows_to_delete = []
    for i, row in enumerate(grid):
        if contains_run_length(row, length):
            rows_to_delete.append(i)
    grid = np.delete(grid, rows_to_delete, axis=0)

valid_rows = np.copy(grid)
valid_rows = ~valid_rows  # True is black, False is white
valid_rows = valid_rows[np.sum(valid_rows, axis=1) < 13]  # Remove all-black row
flipped_valid_rows = np.flip(valid_rows, axis=1)  # Precompute flipped rows for later

<IPython.core.display.Javascript object>

In [6]:
# Compute all symmetric rows

symmetric_row_indexes = []
for i, row in enumerate(grid):
    if all(row == np.flip(row)):
        symmetric_row_indexes.append(i)

symmetric_rows = np.copy(grid[symmetric_row_indexes])
symmetric_rows = ~symmetric_rows  # True is black, False is white
symmetric_rows = symmetric_rows[
    np.sum(symmetric_rows, axis=1) < 13
]  # Remove all-black row

<IPython.core.display.Javascript object>

In [7]:
def row_is_valid(grid, row_num):
    (word_starts_indexes,) = np.where(
        np.append(
            [np.logical_not(grid[row_num][0])], np.diff(grid[row_num].astype(int)) == -1
        )
    )
    (word_ends_indexes,) = np.where(
        np.append(
            np.diff(grid[row_num].astype(int)) == 1, [np.logical_not(grid[row_num][-1])]
        )
    )

    for word_start_index, word_end_index in zip(word_starts_indexes, word_ends_indexes):
        subgrid = grid[:, word_start_index : word_end_index + 1]
        if 0 < row_num < grid.shape[0] - 1:
            unchecked_squares = np.logical_and(*subgrid[[row_num - 1, row_num + 1]])
        elif row_num == 0:
            unchecked_squares = subgrid[1]
        elif row_num == grid.shape[0] - 1:
            unchecked_squares = subgrid[grid.shape[0] - 2]

        expected_num_unchecked = (word_end_index - word_start_index) / 2

        has_expected_num_unchecked = (
            sum(unchecked_squares) == np.ceil(expected_num_unchecked)
            if ROUND_UP_UNCHECKED
            else (
                np.floor(expected_num_unchecked)
                <= sum(unchecked_squares)
                <= np.ceil(expected_num_unchecked)
            )
        )
        has_max_two_consecutive_unchecked = all(
            [
                sum(group) <= 2
                for bit, group in itertools.groupby(unchecked_squares)
                if bit
            ]
        )
        ends_not_two_consecutive_unchecked = not (
            all(unchecked_squares[:2]) or all(unchecked_squares[-2:])
        )

        if not (
            has_expected_num_unchecked
            and has_max_two_consecutive_unchecked
            and ends_not_two_consecutive_unchecked
        ):
            return False

    return True

<IPython.core.display.Javascript object>

In [8]:
def grid_from_indexes(*indexes):
    return np.vstack(
        [valid_rows[i] for i in indexes[:-1]]
        + [symmetric_rows[indexes[-1]]]
        # Make sure to reverse the flipped rows!
        + [flipped_valid_rows[i] for i in indexes[:-1]][::-1]
    )


def translate(row):
    return "".join(["⬛" if x else "⬜" for x in row])


def visualize(grid):
    print("\n".join([translate(row) for row in grid]))

<IPython.core.display.Javascript object>

In [9]:
with open("valid_middle_rows_indexes.json", "r") as f:
    valid_middle_rows_indexes = json.load(f)
print(len(valid_middle_rows_indexes))

7133


<IPython.core.display.Javascript object>

In [10]:
valid_grids_indexes = []
valid_middle_rows_indexes = [valid_middle_rows_indexes[-1]]  # FIXME: remove me!

set_valid_rows = set([translate(row) for row in valid_rows])

while valid_middle_rows_indexes:
    indexes = valid_middle_rows_indexes.pop()

    (must_be_whites_indexes,) = np.where(
        (grid_from_indexes(*indexes)[:3].T == np.array([True, False, False])).all(
            axis=1
        )
    )
    (valid_rows_to_search_indexes,) = np.where(
        valid_rows[:, must_be_whites_indexes].all(axis=1)
    )

    for i in valid_rows_to_search_indexes:
        grid = grid_from_indexes(i, *indexes)
        if row_is_valid(grid, 0):
            if len([i, *indexes]) == GRID_LENGTH // 2 + 1:
                if all(
                    [
                        translate(row) in set_valid_rows
                        for row in np.rot90(grid)[: GRID_LENGTH // 2 + 1]
                    ]
                ):
                    valid_grids_indexes.append([i, *indexes])
            else:
                valid_middle_rows_indexes.append([i, *indexes])

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [11]:
len(valid_grids_indexes)

37386

<IPython.core.display.Javascript object>